# Import Libraries 

In [16]:
import pandas as pd
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy import units as u
import os
import matplotlib.pyplot as plt
import seaborn as sns
from astroquery.simbad import Simbad

# Login to Gaia to access data

In [2]:
# Set the environment variable within the notebook 
os.environ['SSL_CERT_FILE'] = '/usr/local/ciao-4.13/ciao-4.13/ots/lib/python3.7/site-packages/certifi/cacert.pem'

In [3]:
Gaia.login()

INFO: Login to gaia TAP server [astroquery.gaia.core]
User: jperea
Password: ········
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]


# Read in X-ray file path and create data frame
This file is the X-ray catalog of sources in IC 342 that we found at a probability of 99%. 

142 Sources. 
This only includes the X-ray sources that are within our HST observation. 

The columns are source name, ra, dec, -, -, sig1, sig2, sig3. 
sig1,2, and 3 refer to the positional uncertainty of Chandra at 1, 2, and 3 sigma confidence intervals. 

In [4]:
file_path_Xray = 'Xray99_final.txt'

In [5]:
df_Xray = pd.read_csv(file_path_Xray, sep=',', header=None)

### Change column names 

In [6]:
column_names_Xray = ['source', 'ra', 'dec', '?', '?', 'sig1', 'sig2', 'sig3']

In [7]:
df_Xray.columns = column_names_Xray

In [8]:
# dropping two of the columns we do not need
drop_col_Xray = [col for col in df_Xray.columns if '?' in col]

df_Xray = df_Xray.drop(columns=drop_col_Xray)

In [9]:
df_Xray

,source,ra,dec,sig1,sig2,sig3
0,034642.35+680938.2,56.676577,68.160591,2.561205,5.122410,7.683615
1,034643.19+680211.6,56.680053,68.036526,2.471818,4.943636,7.415454
2,034702.73+681001.6,56.761489,68.167083,2.316915,4.633831,6.950746
3,034644.91+680859.1,56.687242,68.149708,2.210881,4.421761,6.632642
4,034619.81+680837.5,56.582643,68.143720,2.182250,4.364499,6.546749
...,...,...,...,...,...,...
137,034652.81+680504.5,56.720169,68.084545,0.493928,0.987855,1.481783
138,034559.57+680538.0,56.498318,68.093858,0.493555,0.987110,1.480664
139,034648.44+680547.2,56.701931,68.096418,0.483667,0.967335,1.451002
140,034643.65+680611.3,56.681968,68.103121,0.482793,0.965587,1.448380


# Query Gaia

We want to compare our positions in the X-ray catalog to positions in the Gaia database to see if we have any foreground stars in our X-ray list. 

In [10]:
def query_gaia_adql(ra, dec, radius):
    """
    Queries the Gaia catalog for sources around a given position (RA, Dec) within a specified radius using ADQL.

    Parameters:
    ra (float): Right Ascension in degrees.
    dec (float): Declination in degrees.
    radius (float): Search radius in arcseconds.

    Returns:
    astropy.table.Table: Results from the Gaia query.
    """
    # Convert radius from arcseconds to degrees
    radius_deg = radius / 3600.0

    # Construct ADQL query
    query = f"""
    SELECT * FROM gaiadr3.gaia_source
    WHERE DISTANCE(POINT({ra}, {dec}), POINT(ra, dec)) < {radius_deg}
    """
    
    # Perform the query using Astroquery's Gaia module
    job = Gaia.launch_job_async(query)
    result = job.get_results()
    
    # Return the result of the query
    return result

In [11]:
# Initialize a list to hold the results from Gaia
all_results = []

# Loop through each row in the DataFrame
for index, row in df_Xray.iterrows():
    # Extract the RA, Dec, and search radius (sig3) for the current row
    source_id = row['source']
    ra = row['ra']
    dec = row['dec']
    radius = row['sig3']
    
    # Query Gaia using the current row's parameters
    result = query_gaia_adql(ra, dec, radius)
    
    # Convert result to pandas DataFrame
    df_result = result.to_pandas()
    
    # Add a column to the DataFrame to indicate the source identifier
    df_result['query_source'] = source_id
    
    # Append the DataFrame to the list
    all_results.append(df_result)

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


## Move all results into a data frame

In [12]:
# Concatenate all the individual DataFrames into one DataFrame
df_Gaia_results = pd.concat(all_results, ignore_index=True)


In [32]:
# Print all column names
for column in df_Gaia_results.columns:
    print(column)

solution_id
DESIGNATION
SOURCE_ID
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

## Filter sources with parallax 

If Gaia is able to measure parallax then the source had to be measured within the Galaxy. 

In [47]:
# Filter sources with non-zero parallax
measurable_parallax_df = df_Gaia_results[df_Gaia_results['parallax'] > 0]

# Display the first few rows of the filtered DataFrame
measurable_parallax_df

,solution_id,DESIGNATION,SOURCE_ID,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,query_source
0,1636148068921376768,Gaia DR3 493584311742730240,493584311742730240,638876494,2016.0,56.675081,0.308686,68.159966,0.389643,0.974275,0.572464,1.701897,1.604861,1.524500,0.329863,0.501476,0.494530,-0.165374,-0.028232,-0.215916,0.017589,0.057571,0.015098,-0.120402,0.102727,0.100845,-0.176036,470,0,465,5,-0.870766,462.309601,0.000000,0.000000e+00,95,False,NaN,1.352684,0.113283,0.238218,0.018687,-0.002088,-0.149766,0.064973,56,28,0.695117,61,23,0,0.034843,117.171356,0,0,0.970679,0.192321,0.245008,0.105517,0.397964,23.025877,72.561600,-26.534924,-20.979025,False,498,134.206270,0.605606,221.606613,20.367935,40,49.313850,5.260668,9.374065,21.106119,53,99.424940,5.875319,16.922474,19.754158,1.108285,0,0,0,1,0,1.351961,0.738184,0.613777,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NOT_AVAILABLE,138.123849,10.623278,72.656042,46.717630,False,False,0,False,False,False,False,False,False,False,False,7.264040e-09,2.822788e-10,9.999993e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034642.35+680938.2
2,1636148068921376768,Gaia DR3 493477762194779392,493477762194779392,903113288,2016.0,56.602521,0.870764,68.052598,1.004939,3.915352,1.566494,2.499

## What are the sources?

We can look at 'DESIGNATION', 'SOURCE_ID', 'ra', 'dec', 'parallax', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'pmra', 'pmdec' to get an idea of what the source is.

In [38]:
# Select key columns to identify the sources
key_columns = [
    'DESIGNATION', 'SOURCE_ID', 'ra', 'dec', 'parallax', 
    'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 
    'pmra', 'pmdec'
]

# Create a new DataFrame for the key columns to avoid SettingWithCopyWarning
df_key_info_Gaia = df_Gaia_results.loc[:, key_columns]

# Display the selected columns for the first few rows
print(df_key_info_Gaia.head())

                   DESIGNATION           SOURCE_ID         ra        dec  \
0  Gaia DR3 493584311742730240  493584311742730240  56.675081  68.159966   
1  Gaia DR3 493584105585319680  493584105585319680  56.777327  68.158333   
2  Gaia DR3 493477762194779392  493477762194779392  56.602521  68.052598   
3  Gaia DR3 493583968146334592  493583968146334592  56.744889  68.130734   
4  Gaia DR3 493476868839800320  493476868839800320  56.765787  68.015351   

   parallax  phot_g_mean_mag  phot_bp_mean_mag  phot_rp_mean_mag      pmra  \
0  0.974275        20.367935         21.106119         19.754158  1.524500   
1 -2.197080        20.917095         21.167871         20.732855 -0.679687   
2  3.915352        20.838167         21.151302         20.055437  1.082188   
3  0.400624        20.176601         21.155083         19.640652  0.707503   
4  1.944744        19.830391               NaN               NaN -2.441091   

       pmdec  
0   0.501476  
1   3.122857  
2  -3.982870  
3   0.563839  

### Calculate the distance

Since we know the parallax, we can calculate the distance to the source and see if it is a foreground object or something in IC 342. 

$$ \rm Distance \ (pc) = \frac{1}{Parallax \ (arcseconds)} $$

Since the parallax values from Gaia are provided in milliarcseconds (mas), we need to convert them to arcseconds before calculating the distance. 

$\rm Parallax \ (arcseconds) = Parallax \ (mas) \times 10^{-3}$

In [39]:
# Define a function to calculate the distance in parsecs from parallax in milliarcseconds
def calculate_distance(parallax_mas):
    """
    Calculate the distance in parsecs from parallax in milliarcseconds.
    Returns NaN if parallax is zero or negative.
    
    Parameters:
    parallax_mas (float): Parallax value in milliarcseconds.
    
    Returns:
    float: Distance in parsecs.
    """
    # Convert parallax from milliarcseconds to arcseconds
    parallax_arcsec = parallax_mas * 1e-3  # 1 mas = 1e-3 arcseconds
    
    # Check if parallax is zero or negative
    if parallax_arcsec <= 0:
        return float('nan')  # Return NaN for invalid parallax values
    
    # Calculate the distance in parsecs using the formula distance = 1 / parallax
    distance_parsecs = 1 / parallax_arcsec
    
    return distance_parsecs  # Return the calculated distance

In [40]:
# Apply the distance calculation to the DataFrame using .loc to avoid SettingWithCopyWarning
df_key_info_Gaia.loc[:, 'distance_parsecs'] = df_key_info_Gaia['parallax'].apply(calculate_distance)

# Display the DataFrame with the new distance column
df_key_info_Gaia

,DESIGNATION,SOURCE_ID,ra,dec,parallax,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,pmra,pmdec,distance_parsecs
0,Gaia DR3 493584311742730240,493584311742730240,56.675081,68.159966,0.974275,20.367935,21.106119,19.754158,1.524500,0.501476,1026.403816
1,Gaia DR3 493584105585319680,493584105585319680,56.777327,68.158333,-2.197080,20.917095,21.167871,20.732855,-0.679687,3.122857,NaN
2,Gaia DR3 493477762194779392,493477762194779392,56.602521,68.052598,3.915352,20.838167,21.151302,20.055437,1.082188,-3.982870,255.404887
3,Gaia DR3 493583968146334592,493583968146334592,56.744889,68.130734,0.400624,20.176601,21.155083,19.640652,0.707503,0.563839,2496.103469
4,Gaia DR3 493476868839800320,493476868839800320,56.765787,68.015351,1.944744,19.830391,NaN,NaN,-2.441091,-10.517910,514.206484
5,Gaia DR3 493476868842805248,493476868842805248,56.767141,68.015481,0.161899,18.727898,19.366577,17.825161,0.726282,0.208434,6176.694327
6,Gaia DR3 493477830915927552,493477830915927552,56.603650,68.058650,NaN,20.867071,20.787893,19.548946,NaN,NaN,NaN
7,Gaia DR3 493477074998225920,493477074998225920,56.749576,68.027043,0.194044,19.223301,19.830351,18.519697,0.519494,0.407186,5153.473045
8,Gaia DR3 493489959900068352,493489959900068352,56.615958,68.093034,0.519766,19.930590,20.678411,19.072666,-1.739144,0.831669,1923.944353
9,Gaia DR3 493477964058645632,493477964058645632,56.678959,68.063364,NaN,21.231934,19.981606,19.025238,NaN,NaN,NaN


IC 342 is 3.45 Mpc away. or 3,450,000 pc. 

We need to see if any of the sources are above. We know they shouldn't be because Gaia measures parallax within the Galaxy (as far as I know). 

In [42]:
# Define the distance to IC 342 in parsecs
distance_IC342 = 3.45e6  # 3,450,000 parsecs

# Calculate the difference in distance from IC 342
df_key_info_Gaia.loc[:, 'distance_diff'] = df_key_info_Gaia['distance_parsecs'] - distance_IC342

# Display the DataFrame with the new distance difference column
print(df_key_info_Gaia[['distance_diff']])

    distance_diff
0   -3.448974e+06
1             NaN
2   -3.449745e+06
3   -3.447504e+06
4   -3.449486e+06
5   -3.443823e+06
6             NaN
7   -3.444847e+06
8   -3.448076e+06
9             NaN
10            NaN
11  -3.441616e+06
12  -3.446993e+06
13  -3.448395e+06
14  -3.445707e+06
15            NaN
16  -3.449156e+06
17            NaN
18  -3.448119e+06
19            NaN
20  -3.449497e+06
21            NaN
22  -3.449130e+06
23            NaN
24  -3.449512e+06
25  -3.447959e+06
26  -3.449753e+06
27  -3.448559e+06
28            NaN
29            NaN
30            NaN
31  -3.449769e+06
32  -3.443691e+06


In [43]:
# Define the threshold distance for the Milky Way in parsecs
milky_way_radius_pc = 30e3  # 30,000 parsecs

# Check if a specific distance is within the Milky Way
def is_within_milky_way(distance_pc):
    return distance_pc <= milky_way_radius_pc

In [45]:
# Check if this distance is within the Milky Way
is_within_galaxy = is_within_milky_way(df_key_info_Gaia['distance_parsecs'])

print(f"Is the source with distance {df_key_info_Gaia['distance_parsecs']} pc within the Milky Way? {is_within_galaxy}")

Is the source with distance 0     1026.403816
1             NaN
2      255.404887
3     2496.103469
4      514.206484
5     6176.694327
6             NaN
7     5153.473045
8     1923.944353
9             NaN
10            NaN
11    8384.427595
12    3007.192503
13    1605.141043
14    4293.294162
15            NaN
16     843.607908
17            NaN
18    1880.635216
19            NaN
20     502.875353
21            NaN
22     869.778974
23            NaN
24     488.145819
25    2040.701914
26     246.731171
27    1440.657862
28            NaN
29            NaN
30            NaN
31     230.606096
32    6309.395454
Name: distance_parsecs, dtype: float64 pc within the Milky Way? 0      True
1     False
2      True
3      True
4      True
5      True
6     False
7      True
8      True
9     False
10    False
11     True
12     True
13     True
14     True
15    False
16     True
17    False
18     True
19    False
20     True
21    False
22     True
23    False
24     True
25     True
26

It is clear that these are nowhere near IC 342, so none of these sources are sources within IC 342. 


## Create region file 

Want to see where these sources are in the HST field

In [28]:
# Create a region file
region_file_path = 'Gaia_sources.reg'

with open(region_file_path, 'w') as file:
    # Write the header
    file.write('# Region file format: DS9 version 4.1\n')
    file.write('global color=purple dashlist=8 3 width=1 font="helvetica 10 normal roman" ')
    file.write('select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n')
    file.write('fk5\n')
    
    # Write the coordinates and query_source text for each source
    for index, row in measurable_parallax_df.iterrows():
        ra = row['ra']
        dec = row['dec']
        query_source_text = row['query_source']
        file.write(f'point({ra},{dec}) # point=x text={{{query_source_text}}}\n')

print(f'Region file created: {region_file_path}')

Region file created: Gaia_sources.reg


# Query Simbad

In [25]:
# Define a function to query Simbad
def query_simbad(ra, dec, radius_arcsec):
    """
    Queries the Simbad catalog for sources around a given position (RA, Dec) within a specified radius.

    Parameters:
    ra (float): Right Ascension in degrees.
    dec (float): Declination in degrees.
    radius_arcsec (float): Search radius in arcseconds.

    Returns:
    astropy.table.Table: Results from the Simbad query.
    """
    coord = SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='icrs')
    result_table = Simbad.query_region(coord, radius=radius_arcsec * u.arcsec)
    return result_table

In [29]:
# Initialize a list to hold the results from Simbad
all_simbad_results = []

# Create a region file
region_file_path = 'simbad_sources.reg'

with open(region_file_path, 'w') as file:
    # Write the header
    file.write('# Region file format: DS9 version 4.1\n')
    file.write('global color=orange dashlist=8 3 width=1 font="helvetica 10 normal roman" ')
    file.write('select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n')
    file.write('fk5\n')
    
    # Query Simbad for each source and store results
    for index, row in df_Xray.iterrows():
        ra = row['ra']
        dec = row['dec']
        radius_arcsec = row['sig3']  # Use the 3-sigma radius for the query
        query_source_text = row['source']
        simbad_result = query_simbad(ra, dec, radius_arcsec)
        
        if simbad_result is not None:
            # Convert Simbad results to DataFrame and add query_source column
            df_simbad_result = simbad_result.to_pandas()
            df_simbad_result['query_source'] = query_source_text
            
            # Append the DataFrame to the list
            all_simbad_results.append(df_simbad_result)
            
            # Write results to the region file
            for simbad_row in simbad_result:
                simbad_ra = simbad_row['RA']
                simbad_dec = simbad_row['DEC']
                simbad_coord = SkyCoord(simbad_ra, simbad_dec, unit=(u.hourangle, u.deg))
                file.write(f'point({simbad_coord.ra.deg},{simbad_coord.dec.deg}) # point=x text={{{query_source_text}}}\n')

In [27]:
# Concatenate all the individual DataFrames into one DataFrame
if all_simbad_results:
    df_all_simbad_results = pd.concat(all_simbad_results, ignore_index=True)
else:
    df_all_simbad_results = pd.DataFrame()

# Display the first few rows of the combined DataFrame
print(df_all_simbad_results.head())

print(f'Region file created: {region_file_path}')

                   MAIN_ID             RA            DEC  RA_PREC  DEC_PREC  \
0      NVSS J034728+680825     03 47 28.3      +68 08 22        5         5   
1            [VAU2013] 101    03 46 22.60    +68 03 00.4        6         6   
2       [HC2009b] IC 342-2    03 46 27.89    +68 05 33.9        6         6   
3            [BBL2003b] 15  03 46 26.4039  +68 04 54.598       14        14   
4  ZTF J034629.87+680112.9    03 46 29.88    +68 01 12.9        6         6   

   COO_ERR_MAJA  COO_ERR_MINA  COO_ERR_ANGLE COO_QUAL COO_WAVELENGTH  \
0     4167.0000      4167.000              0        D              R   
1           NaN           NaN              0        D              O   
2           NaN           NaN              0        D              O   
3        0.0062         0.008             90        A              O   
4           NaN           NaN              0        C              O   

           COO_BIBCODE  SCRIPT_NUMBER_ID        query_source  
0  2017A&A...598A..78I       

In [46]:
df_all_simbad_results

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID,query_source
0,NVSS J034728+680825,03 47 28.3,+68 08 22,5,5,4167.0000,4167.0000,0,D,R,2017A&A...598A..78I,1,034728.98+680823.6
1,[VAU2013] 101,03 46 22.60,+68 03 00.4,6,6,NaN,NaN,0,D,O,2013SerAJ.187...11V,1,034622.57+680302.3
2,[HC2009b] IC 342-2,03 46 27.89,+68 05 33.9,6,6,NaN,NaN,0,D,O,2009ApJ...703..894H,1,034627.53+680534.9
3,[BBL2003b] 15,03 46 26.4039,+68 04 54.598,14,14,0.0062,0.0080,90,A,O,2020yCat.1350....0G,1,034626.41+680454.5
4,ZTF J034629.87+680112.9,03 46 29.88,+68 01 12.9,6,6,NaN,NaN,0,C,O,2020ApJS..249...18C,1,034629.81+680113.5
5,[MPK2008] C19,03 47 10.9589,+68 02 52.134,14,14,0.0386,0.0505,90,A,O,2020yCat.1350....0G,1,034711.02+680252.0
6,[VAU2013] 188,03 47 04.60,+68 02 06.9,6,6,NaN,NaN,0,D,O,2013SerAJ.187...11V,1,034704.29+680206.1
7,USNO-A2.0 1575-01802769,03 46 25.9774,+68 04 19.844,14,14,0.0173,0.0225,90,A,O,2020yCat.1350....0G,1,034625.94+680419.6
8,EQ J034632+680356,03 46 32.90,+68 03 56.0,6,6,NaN,NaN,0,D,,2003MNRAS.342..709R,1,034632.82+680356.5
9,XMMU J034622.4+680505,03 46 22.170,+68 05 05.84,7,7,NaN,NaN,0,D,,2008ApJ...686..995M,1,034622.18+680505.8
